# Initialization:

In [1]:
import torch
import dqc
import dqc.xc
import dqc.utils

from DQCAdapter import DQCAdapter

In [2]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p
            elif number_of_parameter == 1: # parameter p
                return self.a * rho ** (self.p - 1)

In [3]:
a = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.double))
p = torch.nn.Parameter(torch.tensor(2.0, dtype=torch.double))
myxc = MyLDAX(a, p)

In [4]:
mol = dqc.Mol(moldesc="N -1 0 0; N 1 0 0", basis="3-21G")
qc = dqc.KS(mol, xc=myxc).run()
ene = qc.energy()
print(ene)

tensor(-54.0932, dtype=torch.float64, grad_fn=<AddBackward0>)


/home/radium/miniconda3/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=2.508e-02, |f|=2.288e-01 at iter 32
  warnings.warn(ConvergenceWarning(msg))


In [5]:
adapter = DQCAdapter(qc)

In [6]:
fockian = adapter.get_fockian()
print(fockian.size())

torch.Size([18, 18])


In [7]:
coefficients = adapter.get_orbital_coefficients()
coefficients.size()

torch.Size([18, 7])

In [8]:
orbital_energies = adapter.get_orbital_energies()
orbital_energies.size()

torch.Size([7])

In [9]:
occupancy = adapter.get_orbital_occupancy()
occupancy.size()

torch.Size([7])

In [10]:
number_of_occupied_orbitals = adapter.get_number_of_occupied_orbitals()

# Calculating adjoint derivatives:

## Derivatives from energy:

### Calculating $\frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}}$

#### The first way to calculate derivative from energy with respect to coefficients:
Every type of orbitals can be used

$$ \frac{\partial E[\rho](\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij}$$
so
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\textbf{C}\textbf{F}$$
$\textbf{f}$ here is matrix: $f_{ab}=\delta_{ab}f_a$

In [11]:
dE_wrt_dC_first_way = 2 * torch.einsum("b,ib,ij->bj", occupancy, coefficients, fockian)
dE_wrt_dC_first_way.size()

torch.Size([7, 18])

#### The second way to calculate derivative from energy with respect to coefficients
Only canonical (i.e. eigenfunctions of fockian) orbitals can be used

from the other hand, 
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij} = 2f_b \epsilon_b C_{bj}$$
so
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\epsilon\textbf{C}$$
$\epsilon$ here is matrix: $\epsilon_{ab}=\delta_{ab}\epsilon_a$

In [12]:
dE_wrt_dC_second_way = 2 * torch.einsum("b,b,jb->bj", occupancy, orbital_energies, coefficients)
dE_wrt_dC_second_way.size()

torch.Size([7, 18])

Check that both ways lead to the same results:

In [13]:
print(torch.linalg.matrix_norm(dE_wrt_dC_first_way - dE_wrt_dC_second_way))

tensor(2.7748e-13, dtype=torch.float64, grad_fn=<CopyBackwards>)


### Calculating $\frac{\partial E[\rho](\vec{\theta})}{\partial \vec{\epsilon}}$

$$\frac{\partial E[\rho](\vec{\theta})}{\partial \epsilon_{b}} = 0$$ for all $\epsilon_b$. So:

In [14]:
dE_wrt_depsilon = torch.zeros(number_of_occupied_orbitals)
dE_wrt_depsilon.size()

torch.Size([7])

## Derivatives from normalization equations:

### Calculating $\frac{\partial \vec{r}(\textbf{C})}{\partial \textbf{C}}$

$$\frac{\partial r_{a}(\textbf{C})}{\partial C_{ck}} = 2\delta_{ac}C_{ck}$$


In [15]:
occupied_orbitals_kronecker = torch.eye(number_of_occupied_orbitals)

In [19]:
dnorm_wrt_dC = 2 * torch.einsum("ac,kc->akc", occupied_orbitals_kronecker, coefficients)
dnorm_wrt_dC.size()

torch.Size([7, 18, 7])

### Calculating $\frac{\partial \vec{r}(\textbf{C})}{\partial \vec{\epsilon}}$

$$\frac{\partial r_{a}(\textbf{C})}{\partial \epsilon_b} = 0$$


In [18]:
dnorm_wrt_depsilon = torch.zeros((number_of_occupied_orbitals, number_of_occupied_orbitals))
dnorm_wrt_depsilon.size()

torch.Size([7, 7])

## Derivatives from Roothan equations:

### Calculating $\frac{\partial \textbf{r}(\textbf{C};\;\vec{\theta})}{\partial \textbf{C}}$

$$G_{Cou, ijkl} = \iint b_i(\vec{r}) \frac{b_k(\vec{r'})b_l(\vec{r'})}{|\vec{r}-\vec{r'}|} b_j(\vec{r})d\vec{r'}d\vec{r}$$
$$G_{XC, ijkl} = 
\int b_i(\vec{r}) b_k(\vec{r}) \frac{\partial V_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \rho(\vec{r})}b_l(\vec{r})b_j(\vec{r})d\vec{r}$$
$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial C_{ck}} = 
(F_{ik}[\rho](\vec{\theta}) - \epsilon_c\delta_{ik})\delta_{ac} +
2f_c\sum_j \sum_{l}C_{cl}\left(G_{Cou,ijkl} + G_{XC,ijkl}\right)C_{aj}
$$

First term:

In [24]:
number_of_all_orbitals = adapter.get_number_of_all_orbitals()
all_orbitals_kronecker = torch.eye(number_of_all_orbitals)
dRoothan_wrt_dC_first_first_term = torch.einsum("ik,ac->iack", fockian, occupied_orbitals_kronecker)
dRoothan_wrt_dC_first_second_term = torch.einsum("c,ik,ac->iack", orbital_energies, all_orbitals_kronecker, occupied_orbitals_kronecker)
dRoothan_wrt_dC_first_term = dRoothan_wrt_dC_first_first_term + dRoothan_wrt_dC_first_second_term
dRoothan_wrt_dC_first_term.size()

torch.Size([18, 7, 7, 18])

Second term:
(NB: fourDtensor loses symmetry of XC-tensor):

In [26]:
fourDtensor = adapter.get_four_center_elrep_tensor() + adapter.get_four_center_xc_tensor()
dRoothan_wrt_dC_second_term = 2 * torch.einsum("c,ijkl,lc,ja->iack", occupancy, fourDtensor, coefficients, coefficients)
dRoothan_wrt_dC_second_term.size()

torch.Size([18, 7, 7, 18])

The sum:

In [27]:
dRoothan_wrt_dC = dRoothan_wrt_dC_first_term + dRoothan_wrt_dC_second_term
dRoothan_wrt_dC.size()

torch.Size([18, 7, 7, 18])

### Calculating $\frac{\partial \textbf{r}(\textbf{C};\;\vec{\theta})}{\partial \vec{\epsilon}}$


$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial \epsilon_{c}} = -\delta_{ac}C_{ai}$$

In [20]:
dRoothan_wrt_depsilon = -1 * torch.einsum("ac,kc->akc", occupied_orbitals_kronecker, coefficients)
dRoothan_wrt_depsilon.size()

torch.Size([7, 18, 7])